# balancing dataset

In [1]:
import os
root_dir = os.path.dirname(os.path.abspath("__file__"))
print(root_dir)

d:\Programming\Work\sewerbot_ai


In [2]:
sc_path = os.path.join(root_dir, "src", "pictures", "data")
# sc_path = 'C:\\_TBZ mindenes\\sewerbot_ai\\src\\pictures\\data'
["{}: {}".format(root, len(files)) for root, dirs, files in os.walk(sc_path)]


['d:\\Programming\\Work\\sewerbot_ai\\src\\pictures\\data: 0',
 'd:\\Programming\\Work\\sewerbot_ai\\src\\pictures\\data\\end: 8698',
 'd:\\Programming\\Work\\sewerbot_ai\\src\\pictures\\data\\notEnd: 8698']

In [3]:
min_len = min([len(files) for root, dirs, files in os.walk(sc_path)][1:])
min_len

8698

In [4]:
import random
for root, dirs, files in os.walk(sc_path):
    if (len(files) == 0):
        continue
    del_count = len(files)-min_len
    random.shuffle(files)
    for f in files[:del_count]:
        os.remove(os.path.join(root, f))

In [5]:
["{}: {}".format(root, len(files)) for root, dirs, files in os.walk(sc_path)]

['d:\\Programming\\Work\\sewerbot_ai\\src\\pictures\\data: 0',
 'd:\\Programming\\Work\\sewerbot_ai\\src\\pictures\\data\\end: 8698',
 'd:\\Programming\\Work\\sewerbot_ai\\src\\pictures\\data\\notEnd: 8698']

# train

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

In [7]:
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(15),  # Randomly rotate the image up to 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color
    transforms.RandomGrayscale(p=0.2),  # Convert to grayscale with a probability of 20%
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
data_directory = os.path.join(root_dir, "src", "pictures", "data")
# data_directory = 'C:\\_TBZ mindenes\\sewerbot_ai\\src\\pictures\\data'
dataset = ImageFolder(data_directory, transform=transform)

In [9]:
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=False)

Using cache found in C:\Users\TéBé/.cache\torch\hub\pytorch_vision_main
c:\Users\TéBé\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\TéBé\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [10]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),  
    nn.Linear(256, 2) # binary classification
)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
from tqdm import tqdm
import numpy as np

In [13]:
# batch_size = 144
# batch_size = 144*3
batch_size = 500
data_loader = DataLoader(dataset, batch_size=int(batch_size), shuffle=True)

In [14]:
num_epochs = 30

model_name = 'augmented'
model_path = os.path.join(root_dir, "models", model_name + '.pth')

checkpoint_interval = 5
checkpoint_dir = os.path.join(root_dir, 'models', 'checkpoints')

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.to(DEVICE)
criterion.to(DEVICE)

CrossEntropyLoss()

In [15]:
from utils.utils import save_model, load_model

In [16]:
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(data_loader, desc=f'Epoch {epoch + 1}'):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(data_loader)
    epoch_accuracy = 100 * correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

    if (epoch + 1) % checkpoint_interval == 0 and epoch != num_epochs:
        checkpoint_name = f'{model_name}_checkpoint_{epoch + 1}'
        # checkpoint_path = os.path.join(checkpoint_dir, checkpoint_name)
        # torch.save(model.state_dict(), checkpoint_path)
        # print(f'Checkpoint saved: {checkpoint_name}')
        save_model(epoch, model, optimizer, checkpoint_name, isCheckpoint=True)

# torch.save(model.state_dict(), model_path)
save_model(epoch, model, optimizer, model_name, isCheckpoint=False)

Epoch 1: 100%|██████████| 35/35 [02:10<00:00,  3.73s/it]


Epoch 1/30, Loss: 0.4394, Accuracy: 77.99%


Epoch 2: 100%|██████████| 35/35 [02:01<00:00,  3.47s/it]


Epoch 2/30, Loss: 0.2564, Accuracy: 89.07%


Epoch 3: 100%|██████████| 35/35 [02:01<00:00,  3.47s/it]


...

Epoch 28/30, Loss: 0.0428, Accuracy: 98.21%


Epoch 29: 100%|██████████| 35/35 [02:06<00:00,  3.61s/it]


Epoch 29/30, Loss: 0.0427, Accuracy: 98.25%


Epoch 30: 100%|██████████| 35/35 [02:07<00:00,  3.64s/it]

Epoch 30/30, Loss: 0.0427, Accuracy: 98.21%
=> Saving checkpoint
=> Saving model


In [11]:
torch.save(model.state_dict(), model_path)